<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=25bc272eab63c1bf2229229a442361c87b1defc787c95d0e5f2685234266dae8
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1


    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-03 10:10:42
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.41 C
-------------------
Today PnL: -93.08 K (-0.65%)
Current PnL: -24.30 L (-15.93%)
CY Booked + Current PnL: -10.41 L (-6.82%)
-------------------
Total profit:  1.37 L
Total loss:  -25.67 L
-------------------
Total Booked + Current PnL: 16.57 L (13.19%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.82%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 89.71 L (63.41%)
Deployed:  1.26 C
Current:  1.41 C
CAGR/XIRR %: 6.67%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,-0.29,-16.79,20.29,0.09,17014.0,-16924.0,83853.0,94.66,37.0,M-SC,2.92,253.0,-0.99,0.59,10.16,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-1.05,-10.09,11.24,0.02,17641.0,-17612.0,156947.0,139.40,62.0,M-SC,14.17,234.0,-1.00,1.11,36.52,OX40N,NTT,PAINTS
51,MASFIN,398.61,-0.09,-2.65,25.37,22.05,24199.0,-2595.0,95385.0,-15.99,58.0,H-SC,6.71,164.0,-0.11,0.67,38.65,XR,ATH,FINANCE
43,ITC,452.00,-0.27,-2.30,13.04,10.44,25497.0,-4611.0,195527.0,-41.56,39.0,X-LC,1.52,5.0,-0.18,1.38,3.13,X40,NTT,FMCG
35,ICICIGI,2252.93,-1.66,4.23,15.32,20.19,27836.0,7374.0,181694.0,-18.38,42.0,X-MC,6.67,68.0,0.26,1.28,19.98,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,ROUTE,2227.21,11.00,-43.70,200.28,69.05,150034.0,-58154.0,74912.0,-54.32,63.0,H-SC,23.56,140.0,-0.39,0.53,15.53,SR,ATH,IT
13,BSOFT,831.70,2.25,-24.93,100.87,50.79,106502.0,-35070.0,105583.0,-3.75,69.0,H-SC,5.92,151.0,-0.33,0.75,22.93,XR,ATH,IT
85,WIPRO,420.00,1.77,4.43,64.96,72.27,114945.0,7499.0,176947.0,-10.16,66.0,M-LC,3.13,99.0,0.07,1.25,10.87,XR,NTT,IT
9,BAJAJHFL,181.50,1.22,-19.68,84.83,48.45,136617.0,-39458.0,161048.0,-25.46,20.0,X-MC,8.80,64.0,-0.29,1.14,1.22,X40N,ATH,FINANCE
67,SIS,528.00,1.21,-24.37,60.34,21.26,50669.0,-27060.0,83972.0,1980.83,47.0,H-SC,3.90,166.0,-0.53,0.59,13.45,OX40N,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.0,-2.99,-37.99,82.07,12.90,111197.0,-83005.0,135491.0,99.52,49.0,H-SC,12.08,161.0,-0.75,0.96,12.66,OX40N,NTT,FINANCE
24,EASEMYTRIP,26.4,-2.46,-54.03,216.93,45.70,176329.0,-95531.0,81284.0,-15.63,64.0,M-SC,23.37,205.0,-0.54,0.57,17.16,XY24,NTT,TRAVEL
84,WHIRLPOOL,2270.0,-2.28,-20.91,130.01,81.92,118043.0,-24003.0,90795.0,-53.62,22.0,M-SC,4.95,236.0,-0.20,0.64,7.49,XR,NTT,DURABLES
52,MEDANTA,1486.0,-2.19,0.01,24.06,24.08,30548.0,17.0,126967.0,-8.06,38.0,X-SC,5.85,89.0,0.00,0.90,19.63,XY24,NTT,HEALTHCARE
27,GLAXO,3466.2,-1.97,-0.85,40.33,39.15,79692.0,-1684.0,197600.0,-20.38,39.0,X-MC,11.27,60.0,-0.02,1.40,27.91,X40,ATH,PHARMA


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.21,0.17,106.38,106.72,131425.0,207.0,123543.0,-50.44,40.0,H-SC,1.73,139.0,0.00,0.87,23.59,AR,ATH,MISC
37,IEX,219.00,-1.63,-0.83,49.88,48.63,98905.0,-1669.0,198285.0,-34.11,59.0,H-SC,14.82,136.0,-0.02,1.40,10.62,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.29,-5.93,121.02,107.91,168079.0,-8759.0,138885.0,-23.39,33.0,M-SC,11.14,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.05,-10.09,11.24,0.02,17641.0,-17612.0,156947.0,139.40,62.0,M-SC,14.17,234.0,-1.00,1.11,36.52,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.29,-16.79,20.29,0.09,17014.0,-16924.0,83853.0,94.66,37.0,M-SC,2.92,253.0,-0.99,0.59,10.16,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.28,-23.24,47.82,13.47,98992.0,-62658.0,207009.0,-68.51,31.0,H-SC,2.16,158.0,-0.63,1.46,5.26,XY24,NTT,PAINTS
67,SIS,528.00,1.21,-24.37,60.34,21.26,50669.0,-27060.0,83972.0,1980.83,47.0,H-SC,3.90,166.0,-0.53,0.59,13.45,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.77,4.43,64.96,72.27,114945.0,7499.0,176947.0,-10.16,66.0,M-LC,3.13,99.0,0.07,1.25,10.87,XR,NTT,IT
38,INDIAMART,4810.62,0.21,0.17,106.38,106.72,131425.0,207.0,123543.0,-50.44,40.0,H-SC,1.73,139.0,0.00,0.87,23.59,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.77,4.43,64.96,72.27,114945.0,7499.0,176947.0,-10.16,66.0,M-LC,3.13,99.0,0.07,1.25,10.87,XR,NTT,IT
38,INDIAMART,4810.62,0.21,0.17,106.38,106.72,131425.0,207.0,123543.0,-50.44,40.0,H-SC,1.73,139.0,0.00,0.87,23.59,AR,ATH,MISC
37,IEX,219.00,-1.63,-0.83,49.88,48.63,98905.0,-1669.0,198285.0,-34.11,59.0,H-SC,14.82,136.0,-0.02,1.40,10.62,XR,NTT,MISC
86,ZYDUSLIFE,1286.17,-0.82,-1.40,37.56,35.63,77612.0,-2939.0,206635.0,-15.41,42.0,H-MC,3.64,119.0,-0.04,1.46,15.04,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.29,-5.93,121.02,107.91,168079.0,-8759.0,138885.0,-23.39,33.0,M-SC,11.14,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,0.03,-37.75,117.21,35.20,93875.0,-48579.0,80091.0,4.98,18.0,X-SC,14.25,92.0,-0.52,0.57,0.03,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,1.22,-19.68,84.83,48.45,136617.0,-39458.0,161048.0,-25.46,20.0,X-MC,8.80,64.0,-0.29,1.14,1.22,X40N,ATH,FINANCE
8,AWL,485.00,-0.26,-21.27,94.97,53.50,225607.0,-64195.0,237556.0,-61.84,28.0,X-MC,1.99,58.0,-0.28,1.68,5.51,XY24,NTT,FMCG
23,DMART,5391.45,-1.31,-7.08,37.89,28.12,74075.0,-14907.0,195500.0,-19.86,29.0,X-LC,4.35,19.0,-0.20,1.38,14.87,X40N,ATH,FMCG
55,PGHH,17907.65,-0.68,-5.78,41.97,33.76,79411.0,-11610.0,189210.0,-33.58,29.0,X-MC,4.52,57.0,-0.15,1.34,0.00,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-0.64,-22.67,112.28,64.15,206595.0,-53951.0,184000.0,-54.57,46.0,X-SC,1.45,82.0,-0.26,1.30,3.21,XY24,BTT,CEMENT
43,ITC,452.0,-0.27,-2.30,13.04,10.44,25497.0,-4611.0,195527.0,-41.56,39.0,X-LC,1.52,5.0,-0.18,1.38,3.13,X40,NTT,FMCG
15,CAMS,4762.0,-0.02,0.89,23.69,24.79,60195.0,2237.0,254093.0,-3.69,46.0,X-SC,1.90,86.0,0.04,1.80,24.23,X40N,NTT,MISC
8,AWL,485.0,-0.26,-21.27,94.97,53.50,225607.0,-64195.0,237556.0,-61.84,28.0,X-MC,1.99,58.0,-0.28,1.68,5.51,XY24,NTT,FMCG
36,ICICIPRULI,790.0,-0.83,1.37,29.20,30.97,52313.0,2417.0,179155.0,-22.52,48.0,X-MC,2.18,75.0,0.05,1.27,14.10,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COLPAL,3726.84,-1.63,-19.39,75.55,41.51,160393.0,-51065.0,212300.0,-4.10,33.0,X-MC,7.28,61.0,-0.32,1.50,0.00,XY25,ATH,FMCG
55,PGHH,17907.65,-0.68,-5.78,41.97,33.76,79411.0,-11610.0,189210.0,-33.58,29.0,X-MC,4.52,57.0,-0.15,1.34,0.00,X40,ATH,FMCG
56,QUESS,424.00,-0.18,-29.59,102.87,42.84,47085.0,-19235.0,45771.0,-53.72,38.0,X-SC,37.40,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
11,BATAINDIA,2096.00,0.03,-37.75,117.21,35.20,93875.0,-48579.0,80091.0,4.98,18.0,X-SC,14.25,92.0,-0.52,0.57,0.03,X40,NTT,FOOTWEAR
54,PAGEIND,51605.07,-0.15,-8.27,37.43,26.07,56220.0,-13540.0,150200.0,-32.40,30.0,X-MC,10.46,55.0,-0.24,1.06,0.39,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.18,-29.59,102.87,42.84,47085.0,-19235.0,45771.0,-53.72,38.0,X-SC,37.40,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-0.38,-47.59,190.59,52.31,145008.0,-69076.0,76084.0,-44.25,34.0,X-SC,5.02,91.0,-0.48,0.54,1.84,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.03,-37.75,117.21,35.20,93875.0,-48579.0,80091.0,4.98,18.0,X-SC,14.25,92.0,-0.52,0.57,0.03,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,-2.19,0.01,24.06,24.08,30548.0,17.0,126967.0,-8.06,38.0,X-SC,5.85,89.0,0.00,0.90,19.63,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-1.69,-8.49,32.86,21.58,42140.0,-11898.0,128241.0,-53.90,31.0,X-MC,5.83,56.0,-0.28,0.91,13.53,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.77,-14.14,38.92,19.28,115608.0,-48916.0,297040.0,-25.24,61.0,X-LC,6.74,1.0,-0.42,2.10,9.81,X40,ATH,IT
41,INFY,2275.00,0.06,6.33,45.65,54.87,151880.0,19818.0,332706.0,-17.09,61.0,X-LC,2.68,2.0,0.13,2.35,15.14,X40,BTT,IT
76,TMPV,600.00,-1.24,-15.34,67.95,42.18,159002.0,-42411.0,233999.0,-24.30,32.0,X-LC,3.13,3.0,-0.27,1.65,1.36,XY24,NTT,AUTO
82,VBL,671.64,-1.71,-4.42,41.95,35.67,126634.0,-13972.0,301870.0,-15.77,55.0,X-LC,5.41,4.0,-0.11,2.13,8.82,X40N,ATH,FMCG
43,ITC,452.00,-0.27,-2.30,13.04,10.44,25497.0,-4611.0,195527.0,-41.56,39.0,X-LC,1.52,5.0,-0.18,1.38,3.13,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-1.71,-16.35,117.18,81.67,91953.0,-15338.0,78472.0,7050.00,40.0,L-SC,18.68,271.0,-0.17,0.55,51.96,XR,NTT,CERAMICS
51,MASFIN,398.61,-0.09,-2.65,25.37,22.05,24199.0,-2595.0,95385.0,-15.99,58.0,H-SC,6.71,164.0,-0.11,0.67,38.65,XR,ATH,FINANCE
13,BSOFT,831.70,2.25,-24.93,100.87,50.79,106502.0,-35070.0,105583.0,-3.75,69.0,H-SC,5.92,151.0,-0.33,0.75,22.93,XR,ATH,IT
70,SURYODAY,216.00,-0.19,-19.11,52.10,23.03,75467.0,-34221.0,144850.0,57.89,50.0,H-SC,10.00,167.0,-0.45,1.02,43.72,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-1.05,-10.09,11.24,0.02,17641.0,-17612.0,156947.0,139.40,62.0,M-SC,14.17,234.0,-1.00,1.11,36.52,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-1.05,-10.09,11.24,0.02,17641.0,-17612.0,156947.0,139.40,62.0,M-SC,14.17,234.0,-1.00,1.11,36.52,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.36,2.89,17.54,20.95,45438.0,7286.0,259054.0,1.90,77.0,X-LC,17.37,31.0,0.16,1.83,39.56,X40,ATH,PAINTS
13,BSOFT,831.70,2.25,-24.93,100.87,50.79,106502.0,-35070.0,105583.0,-3.75,69.0,H-SC,5.92,151.0,-0.33,0.75,22.93,XR,ATH,IT
50,LTIM,7201.88,-0.11,12.57,16.96,31.66,46994.0,30937.0,277088.0,8.32,68.0,H-LC,14.93,43.0,0.66,1.96,53.74,X200,ATH,IT
66,SIEMENS,4671.50,-0.29,-9.96,39.40,25.51,66019.0,-18535.0,167560.0,2.83,67.0,H-LC,2.74,49.0,-0.28,1.18,23.48,AR,ATH,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.99
1,25,44.75
2,50,75.57


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.42
MC    28.92
LC    25.63
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.61
X40      21.27
X40N     11.91
XR        9.68
AR        9.11
XY25      9.07
OX40N     7.58
X200      1.96
SR        1.03
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.97
X-MC    22.46
X-LC    20.26
M-SC    11.90
X-SC     8.17
H-MC     4.75
H-LC     3.14
L-SC     1.38
M-MC     1.37
M-LC     1.25
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.69,-6.55,41.01
IT,14.12,-13.21,71.66
FINANCE,9.35,-17.14,66.64
MISC,7.43,-23.43,74.00
ELECTRICAL,6.01,-10.33,50.74
PAINTS,5.98,-9.07,25.27
INSURANCE,4.38,-2.07,38.04
PHARMA,3.89,-1.50,33.99
AUTO,3.36,-20.83,72.63


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3130044.0,21
XR,1290980.0,13
AR,1283776.0,10
X40,1029583.0,14
X40N,762920.0,9
OX40N,717629.0,10
XY25,357264.0,6
SR,274663.0,2
MH,77515.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3586076.0,25
M-SC,1374263.0,15
X-MC,1306286.0,16
X-LC,894157.0,11
X-SC,764638.0,8
H-MC,406217.0,3
L-SC,265320.0,3
M-LC,114945.0,1
H-LC,113013.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1246844.0      6
           AR         882038.0      5
M-SC       XY24       782175.0      6
H-SC       XR         774677.0      7
X-MC       X40        463154.0      7
           XY24       389883.0      3
X-LC       X40        384105.0      5
H-SC       OX40N      330339.0      4
M-SC       OX40N      300901.0      5
X-SC       X40N       298086.0      3
           XY24       284228.0      3
H-SC       SR         274663.0      2
X-MC       X40N       264125.0      4
X-LC       XY24       233142.0      2
H-MC       AR         212445.0      2
X-LC       X40N       200709.0      2
H-MC       XY24       193772.0      1
X-MC       XY25       189124.0      2
X-SC       X40        182324.0      2
L-SC       XR         178931.0      2
M-SC       XR         167913.0      2
           AR         123274.0      2
M-LC       XR         114945.0      1
L-SC       OX40N       86389.0      1
H-SC       MH          77515.0      1
X-LC       XY25        76201.0      2
H-LC       AR          66019.0      1
L-MC       XR          54514.0      1
M-MC       XY25        50893.0      1
H-LC       X200        46994.0      1
L-LC       XY25        41046.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
